<a href="https://colab.research.google.com/github/AleksandrSosnin/Big-Data/blob/main/Home_Work2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files


uploaded = files.upload()

Saving AB_NYC_2019.csv to AB_NYC_2019.csv


In [2]:

import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['AB_NYC_2019.csv']))
print(df)

             id                                               name   host_id  \
0          2539                 Clean & quiet apt home by the park      2787   
1          2595                              Skylit Midtown Castle      2845   
2          3647                THE VILLAGE OF HARLEM....NEW YORK !      4632   
3          3831                    Cozy Entire Floor of Brownstone      4869   
4          5022   Entire Apt: Spacious Studio/Loft by central park      7192   
...         ...                                                ...       ...   
48890  36484665    Charming one bedroom - newly renovated rowhouse   8232441   
48891  36485057      Affordable room in Bushwick/East Williamsburg   6570630   
48892  36485431            Sunny Studio at Historical Neighborhood  23492952   
48893  36485609               43rd St. Time Square-cozy single bed  30985759   
48894  36487245  Trendy duplex in the very heart of Hell's Kitchen  68119814   

           host_name neighbourhood_grou

In [3]:
import csv

# Функция маппера
def mapper(file_path):
    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Пропуск заголовка
        for row in reader:
            try:
                price = float(row[9])  # Столбец с ценой (10-й)
                yield 1, price  # Возвращаем единичный счетчик и цену
            except ValueError:
                continue  # Пропуск строк с некорректными данными


In [4]:
# Функция редьюсера
def reducer(mapped_data):
    n = 0
    mean = 0.0
    M2 = 0.0

    for count, price in mapped_data:
        n += count
        delta = price - mean
        mean += delta / n
        M2 += delta * (price - mean)

    if n > 1:
        variance = M2 / (n - 1)
    else:
        variance = 0.0

    return mean, variance


In [5]:
# Путь к загруженному файлу
file_path = 'AB_NYC_2019.csv'

# Шаг 1: Выполняем маппинг
mapped_data = list(mapper(file_path))

# Шаг 2: Выполняем редукцию
mean_price, variance_price = reducer(mapped_data)

# Выводим результаты
print(f"Средняя цена: {mean_price}")
print(f"Дисперсия цены: {variance_price}")


Средняя цена: 152.72068718682823
Дисперсия цены: 57674.02524696077


In [6]:
# Вычисляем среднее и дисперсию с использованием pandas
mean_python = df['price'].mean()
variance_python = df['price'].var()

# Выводим результаты
print(f"Средняя цена (Python): {mean_python}")
print(f"Дисперсия цены (Python): {variance_python}")


Средняя цена (Python): 152.7206871868289
Дисперсия цены (Python): 57674.02524696099


In [ ]:
# SQL-запрос для подсчета средней и дисперсии:
# SELECT AVG(price), VARIANCE(price) FROM airbnb_listings;